In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [12]:
import numpy as np
from scipy.sparse import coo_matrix
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
import scipy.sparse.linalg

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [13]:
def mat_mul_coo(A, B):
    if A.shape[1] != B.shape[0]:
        raise ValueError("Matrix shapes are not compatible for multiplication")

    A = coo_matrix(A)
    B = coo_matrix(B)

    A = A.tocsr()
    B = B.tocsc()

    result_data = []
    result_row = []
    result_col = []

    for i in range(A.shape[0]):
        row = A.getrow(i)
        for j in range(B.shape[1]):
            col = B.getcol(j)
            dot_product = row.dot(col).sum()
            if dot_product != 0:
                result_data.append(dot_product)
                result_row.append(i)
                result_col.append(j)

    result = coo_matrix((result_data, (result_row, result_col)), shape=(A.shape[0], B.shape[1]))

    return result


In [14]:
# Matrix A: a 3x3 matrix with a non-zero diagonal
data = [1, 2, 3]
row = [0, 1, 2]
col = [0, 1, 2]
A = sp.coo_matrix((data, (row, col)), shape=(3, 3))

# Matrix B: a 3x4 matrix with random values
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
row = [0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]
col = [0, 2, 0, 1, 3, 0, 1, 2, 3, 4, 5, 6]
B = sp.coo_matrix((data, (row, col)), shape=(3, 7))

print(mat_mul_coo(A, B))

  (0, 0)	1
  (0, 2)	2
  (1, 0)	6
  (1, 1)	8
  (1, 3)	10
  (2, 0)	18
  (2, 1)	21
  (2, 2)	24
  (2, 3)	27
  (2, 4)	30
  (2, 5)	33
  (2, 6)	36


implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [15]:
import numpy as np
import scipy.sparse as sp

def mat_mul_csr(A, B):
    # transpose B to get the column data easily
    B_transpose = B.transpose()
    result_data = []
    result_indices = []
    result_indptr = [0]

    for i in range(A.shape[0]):
        row_start = A.indptr[i]
        row_end = A.indptr[i + 1]
        row_data = A.data[row_start:row_end]
        row_indices = A.indices[row_start:row_end]

        for j in range(B_transpose.shape[0]):
            col_start = B_transpose.indptr[j]
            col_end = B_transpose.indptr[j + 1]
            col_data = B_transpose.data[col_start:col_end]
            col_indices = B_transpose.indices[col_start:col_end]

            if row_end - row_start == col_end - col_start:
                dot_product = np.inner(row_data, col_data)

                if dot_product != 0:
                    result_data.append(dot_product)
                    result_indices.append(j)
        
        result_indptr.append(len(result_data))

    return sp.csr_matrix((result_data, result_indices, result_indptr), shape=(A.shape[0], B.shape[1]))

# testing the function with an example
data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
row = [0, 0, 1, 1, 1, 2, 2, 2, 2]
col = [0, 2, 0, 1, 3, 0, 1, 2, 3]
A = sp.csr_matrix((data, (row, col)), shape=(3, 4))
B = sp.csr_matrix(np.random.randint(low=0, high=10, size=(4, 3)))
print(mat_mul_csr(A, B))

  (0, 1)	12
  (1, 0)	77
  (1, 2)	74


implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

In [5]:
def solve_lin_sys(A, b):
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix is not square")
    if A.shape[0] != b.shape[0]:
        raise ValueError("Matrix and vector shapes are not compatible")

    lu_factorization = scipy.sparse.linalg.splu(A)
    x = lu_factorization.solve(b)

    return x

In [6]:
# create a sparse matrix in CSR format
data = np.array([1, 2, 3, 4])
row = np.array([0, 0, 1, 2])
col = np.array([0, 1, 1, 2])
A = sp.csr_matrix((data, (row, col)), shape=(3, 3))

# create a vector
b = np.array([1, 2, 3])

x = solve_lin_sys(A, b)
print(x)


[-0.33333333  0.66666667  0.75      ]


/home/olivier/anaconda3/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:347: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
